In [11]:
import pandas as pd
import numpy as np
from numpy import sin, cos, pi
import matplotlib.pyplot as plt
import xlsxwriter as xl
import itertools as it


In [12]:


dat = pd.read_csv(r"F:\CMP\SThota work\Dielectric data\HI-T BTO\BTO-450C-605C-22-07-2022-3am.csv")
dat.head()


,,Unnamed: 1,.1,.2,.3
0,Temp(�C),450,,,
1,Frequency(Hz),C,D,Z,PHAS
2,1.00E+02,9.90E+37,3.27E+00,6.83E+04,-1.41E+01
3,1.45E+02,6.23E-09,2.54E+00,6.18E+04,-2.01E+01
4,2.11E+02,5.05E-09,2.17E+00,6.16E+04,-2.46E+01


In [13]:
def templist(n, s, w, r):
    f = np.array(dat[dat.columns[0]].iloc[n:n+s-3])
    f = [float(i) for i in f]
    n1 = np.array(dat[dat.columns[1]].iloc[n:n+s-3])
    n1 = [float(i) for i in n1]
    n2 = np.array(dat[dat.columns[2]].iloc[n:n+s-3])
    n2 = [float(i) for i in n2]
    n3 = np.array(dat[dat.columns[3]].iloc[n:n+s-3])
    n3 = [float(i) for i in n3]
    n4 = np.array(dat[dat.columns[4]].iloc[n:n+s-3])
    n4 = [float(i) for i in n4]
    eps = [( i * w ) / (pi * (r**2) * 8.8541878128e-12) for i in n1]
    rho = [(j * pi * (r**2)) / w for j in n3]
    Zi = [i * cos(j * pi / 180) for (i,j) in zip(n3, n4)]
    Zii = [i * sin(j * pi / 180) for (i,j) in zip(n3, n4)]
    T = np.array([f, n1, n2, n3, n4, eps, rho, Zi, Zii])
    #print(T)
    
    return T    

In [14]:
def freqset(k, t, t0, h, s, w, r):
    ds = [templist(n+j*s, s, w, r) for j in range(t)]
    n1 = [ds[j][1][k] for j in range(t)]
    n2 = [ds[j][2][k] for j in range(t)]
    n3 = [ds[j][3][k] for j in range(t)]
    n4 = [ds[j][4][k] for j in range(t)]
    eps = [ds[j][5][k] for j in range(t)]
    rho = [ds[j][6][k] for j in range(t)]
    Zi = [ds[j][7][k] for j in range(t)]
    Zii = [ds[j][8][k] for j in range(t)]
    T = [t0 + (i)*h for i in range(t)]
    f = [T, n1, n2, n3, n4, eps, rho, Zi, Zii]
    return f 



In [15]:
n = input("First data point index in excel sheet: ")
n = int(n)
s = input("Cell spacing across temperatures: ")
s = int(s)
t0 = input("Starting temperature: ")
t0 = int(t0)
t = input("No of Temperature points: ")
t = int(t)
h = input("Temperature step size: ")
h = int(h)
r = input("Input the pellet diameter(mm): ")
r = float(r)
r = (r / 2) * 1e-3
w = input("Input pellet width(mm): ")
w = float(w) * 1e-3



First data point index in excel sheet: 2
Cell spacing across temperatures: 33
Starting temperature: 450
No of Temperature points: 32
Temperature step size: 5
Input the pellet diameter(mm): 1
Input pellet width(mm): 1


In [16]:




data = [freqset(k, t, t0, h, s, w, r) for k in range(s-3)]
dataset = [pd.DataFrame(data[j]).T for j in range(s-3)]

for i in range(s-3):
    dataset[i].columns = ['Temp', dat.columns[1], dat.columns[2], dat.columns[3], dat.columns[4], 'Relative permittivity', 'AC Resistivity', 'Z prime', 'Z double prime']
    
with pd.ExcelWriter(r"F:\CMP\SThota work\Dielectric data\HI-T BTO\BTO-450C-605C-22-07-2022-3am-Temp graph.xlsx", engine='xlsxwriter') as writer:
    for k in range(s-3):
        dataset[k].to_excel(writer, sheet_name = dat[dat.columns[0]].iloc[k+2])
        


In [17]:
dataset[k].head()

,Temp,Unnamed: 1,.1,.2,.3,Relative permittivity,AC Resistivity,Z prime,Z double prime
0,450.0,9.900000e+37,9.900000e+37,9.900000e+37,9.900000e+37,1.423628e+52,7.775442e+34,2.242751e+37,9.642617e+37
1,455.0,9.900000e+37,9.900000e+37,9.900000e+37,9.900000e+37,1.423628e+52,7.775442e+34,2.242751e+37,9.642617e+37
2,460.0,9.900000e+37,9.900000e+37,9.900000e+37,9.900000e+37,1.423628e+52,7.775442e+34,2.242751e+37,9.642617e+37
3,465.0,9.900000e+37,9.900000e+37,9.900000e+37,9.900000e+37,1.423628e+52,7.775442e+34,2.242751e+37,9.642617e+37
4,470.0,9.900000e+37,9.900000e+37,9.900000e+37,9.900000e+37,1.423628e+52,7.775442e+34,2.242751e+37,9.642617e+37
